In [1]:
import pandas as pa 


byb=pa.read_csv("Ball_by_Ball.csv")
byb.head()

,Match_Id,Innings_Id,Over_Id,Ball_Id,Team_Batting_Id,Team_Bowling_Id,Striker_Id,Striker_Batting_Position,Non_Striker_Id,Bowler_Id,Batsman_Scored,Extra_Type,Extra_Runs,Player_dissimal_Id,Dissimal_Type,Fielder_Id
0,335987,1,1,1,1,2,1,1,2,14,0,legbyes,1,,,
1,335987,1,1,2,1,2,2,2,1,14,0,,,,,
2,335987,1,1,3,1,2,2,2,1,14,0,wides,1,,,
3,335987,1,1,4,1,2,2,2,1,14,0,,,,,
4,335987,1,1,5,1,2,2,2,1,14,0,,,,,


In [2]:
byb_fil=byb[["Match_Id","Team_Bowling_Id","Bowler_Id","Batsman_Scored","Dissimal_Type"]]
byb_fil.head()

,Match_Id,Team_Bowling_Id,Bowler_Id,Batsman_Scored,Dissimal_Type
0,335987,2,14,0,
1,335987,2,14,0,
2,335987,2,14,0,
3,335987,2,14,0,
4,335987,2,14,0,


In [3]:
byb_fil=byb_fil.rename(columns={"Team_Bowling_Id":"Team_Id","Bowler_Id":"Player_Id"})
byb_fil.head()

,Match_Id,Team_Id,Player_Id,Batsman_Scored,Dissimal_Type
0,335987,2,14,0,
1,335987,2,14,0,
2,335987,2,14,0,
3,335987,2,14,0,
4,335987,2,14,0,


In [4]:
ls=list(byb_fil["Dissimal_Type"].unique())
del ls[0]
del ls[4]
ls.pop()
ls

['caught',
 'bowled',
 'run out',
 'lbw',
 'stumped',
 'caught and bowled',
 'hit wicket']

In [5]:
byb_fil["Dissimal_Type"].value_counts()

                         131058
caught                     3245
bowled                     1028
run out                     583
lbw                         326
stumped                     187
caught and bowled           148
retired hurt                  8
hit wicket                    6
obstructing the field         1
Name: Dissimal_Type, dtype: int64

In [6]:
import numpy as np 

byb_fil["Dissimal_Type"]=[1 if x in ls else 0 for x in byb_fil["Dissimal_Type"]]

In [7]:
byb_fil["Dissimal_Type"].value_counts()

0    131067
1      5523
Name: Dissimal_Type, dtype: int64

In [8]:
byb_fil.head()

,Match_Id,Team_Id,Player_Id,Batsman_Scored,Dissimal_Type
0,335987,2,14,0,0
1,335987,2,14,0,0
2,335987,2,14,0,0
3,335987,2,14,0,0
4,335987,2,14,0,0


In [9]:
byb_fil.dtypes

Match_Id           int64
Team_Id            int64
Player_Id          int64
Batsman_Scored    object
Dissimal_Type      int64
dtype: object

In [10]:
byb_fil["Batsman_Scored"].unique()

array(['0', '4', '6', '1', '2', 'Do_nothing', '5', '3', ' '], dtype=object)

In [11]:
byb_fil["Batsman_Scored"]=np.where(byb_fil["Batsman_Scored"].str.contains("Do"),0,byb_fil["Batsman_Scored"])
byb_fil["Batsman_Scored"]=np.where(byb_fil["Batsman_Scored"].str.contains(" "),0,byb_fil["Batsman_Scored"])

In [12]:
byb_fil["Batsman_Scored"].unique()

array(['0', '4', '6', '1', '2', 0, '5', '3'], dtype=object)

In [13]:
byb_fil["Batsman_Scored"]=pa.to_numeric(byb_fil["Batsman_Scored"])

In [14]:
byb_fil["Batsman_Scored"].unique()

array([0, 4, 6, 1, 2, 5, 3], dtype=int64)

In [15]:
byb_fil.head()

br=byb_fil[["Match_Id","Team_Id","Player_Id","Batsman_Scored"]]
bw=byb_fil[["Match_Id","Team_Id","Player_Id","Dissimal_Type"]]

In [16]:
Runs=br.groupby(["Match_Id","Team_Id","Player_Id"])["Batsman_Scored"].sum()
Wickets=bw.groupby(["Match_Id","Team_Id","Player_Id"])["Dissimal_Type"].sum()

In [17]:
Runs.to_csv("Runs.csv")
Wickets.to_csv("wk.csv")

In [18]:
runs_leaked=pa.read_csv("Runs.csv",names=["Match_Id","Team_Id","Player_Id","Batsman_Scored"],header=None)
wickets_taken=pa.read_csv("wk.csv",names=["Match_Id","Team_Id","Player_Id","Dissimal_Type"],header=None)

In [19]:
runs_leaked.head()

,Match_Id,Team_Id,Player_Id,Batsman_Scored
0,335987,1,1,20
1,335987,1,63,9
2,335987,1,82,21
3,335987,1,84,6
4,335987,1,106,7


In [20]:
wickets_taken.head()

,Match_Id,Team_Id,Player_Id,Dissimal_Type
0,335987,1,1,3
1,335987,1,63,1
2,335987,1,82,3
3,335987,1,84,1
4,335987,1,106,2


In [21]:
runs_wickets=runs_leaked.merge(wickets_taken,on=["Match_Id","Team_Id","Player_Id"],how="outer")

In [22]:
runs_wickets.head()

,Match_Id,Team_Id,Player_Id,Batsman_Scored,Dissimal_Type
0,335987,1,1,20,3
1,335987,1,63,9,1
2,335987,1,82,21,3
3,335987,1,84,6,1
4,335987,1,106,7,2


In [23]:
ls =list(runs_wickets["Player_Id"].unique())
len(ls)

331

In [24]:
players=pa.read_csv("Player.csv")
players.head()

,Player_Id,Player_Name,DOB,Batting_Hand,Bowling_Skill,Country,Is_Umpire,Unnamed: 7
0,1,SC Ganguly,8-Jul-72,Left_Hand,Right-arm medium,India,0,NaN
1,2,BB McCullum,27-Sep-81,Right_Hand,Right-arm medium,New Zealand,0,NaN
2,3,RT Ponting,19-Dec-74,Right_Hand,Right-arm medium,Australia,0,NaN
3,4,DJ Hussey,15-Jul-77,Right_Hand,Right-arm offbreak,Australia,0,NaN
4,5,Mohammad Hafeez,17-Oct-80,Right_Hand,Right-arm offbreak,Pakistan,0,NaN


In [25]:
rw1=runs_wickets.merge(players,on="Player_Id",how="left")
rw1.head()

,Match_Id,Team_Id,Player_Id,Batsman_Scored,Dissimal_Type,Player_Name,DOB,Batting_Hand,Bowling_Skill,Country,Is_Umpire,Unnamed: 7
0,335987,1,1,20,3,SC Ganguly,8-Jul-72,Left_Hand,Right-arm medium,India,0,NaN
1,335987,1,63,9,1,LR Shukla,6-May-81,Right_Hand,Right-arm medium,India,0,NaN
2,335987,1,82,21,3,AB Agarkar,4-Dec-77,Right_Hand,Right-arm fast,India,0,NaN
3,335987,1,84,6,1,I Sharma,2-Sep-88,Right_Hand,Right-arm fast-medium,India,0,NaN
4,335987,1,106,7,2,AB Dinda,25-Mar-84,Right_Hand,Right-arm medium-fast,India,0,NaN


In [26]:
teams=pa.read_csv("Team.csv")
teams.head()

,Team_Id,Team_Name,Team_Short_Code
0,1,Kolkata Knight Riders,KKR
1,2,Royal Challengers Bangalore,RCB
2,3,Chennai Super Kings,CSK
3,4,Kings XI Punjab,KXIP
4,5,Rajasthan Royals,RR


In [27]:
rw2=rw1.merge(teams,on="Team_Id",how="left")

In [28]:
rw2.head()

,Match_Id,Team_Id,Player_Id,Batsman_Scored,Dissimal_Type,Player_Name,DOB,Batting_Hand,Bowling_Skill,Country,Is_Umpire,Unnamed: 7,Team_Name,Team_Short_Code
0,335987,1,1,20,3,SC Ganguly,8-Jul-72,Left_Hand,Right-arm medium,India,0,NaN,Kolkata Knight Riders,KKR
1,335987,1,63,9,1,LR Shukla,6-May-81,Right_Hand,Right-arm medium,India,0,NaN,Kolkata Knight Riders,KKR
2,335987,1,82,21,3,AB Agarkar,4-Dec-77,Right_Hand,Right-arm fast,India,0,NaN,Kolkata Knight Riders,KKR
3,335987,1,84,6,1,I Sharma,2-Sep-88,Right_Hand,Right-arm fast-medium,India,0,NaN,Kolkata Knight Riders,KKR
4,335987,1,106,7,2,AB Dinda,25-Mar-84,Right_Hand,Right-arm medium-fast,India,0,NaN,Kolkata Knight Riders,KKR


In [29]:
rw2=rw2[["Match_Id","Team_Id","Team_Name","Team_Short_Code","Player_Id","Player_Name","Batsman_Scored","Dissimal_Type"]]

In [30]:
rw2.head()

,Match_Id,Team_Id,Team_Name,Team_Short_Code,Player_Id,Player_Name,Batsman_Scored,Dissimal_Type
0,335987,1,Kolkata Knight Riders,KKR,1,SC Ganguly,20,3
1,335987,1,Kolkata Knight Riders,KKR,63,LR Shukla,9,1
2,335987,1,Kolkata Knight Riders,KKR,82,AB Agarkar,21,3
3,335987,1,Kolkata Knight Riders,KKR,84,I Sharma,6,1
4,335987,1,Kolkata Knight Riders,KKR,106,AB Dinda,7,2


In [31]:
match=pa.read_csv("Match.csv")
match.head()

,Match_Id,Match_Date,Team_Name_Id,Opponent_Team_Id,Season_Id,Venue_Name,Toss_Winner_Id,Toss_Decision,IS_Superover,IS_Result,Is_DuckWorthLewis,Win_Type,Won_By,Match_Winner_Id,Man_Of_The_Match_Id,First_Umpire_Id,Second_Umpire_Id,City_Name,Host_Country
0,335987,18-Apr-08,2,1,1,M Chinnaswamy Stadium,2,field,0,1,0,by runs,140.0,1.0,2.0,470,477,Bangalore,India
1,335988,19-Apr-08,4,3,1,"Punjab Cricket Association Stadium, Mohali",3,bat,0,1,0,by runs,33.0,3.0,19.0,471,487,Chandigarh,India
2,335989,19-Apr-08,6,5,1,Feroz Shah Kotla,5,bat,0,1,0,by wickets,9.0,6.0,90.0,472,512,Delhi,India
3,335990,20-Apr-08,7,2,1,Wankhede Stadium,7,bat,0,1,0,by wickets,5.0,2.0,11.0,473,476,Mumbai,India
4,335991,20-Apr-08,1,8,1,Eden Gardens,8,bat,0,1,0,by wickets,5.0,1.0,4.0,474,486,Kolkata,India


In [32]:
rw3=rw2.merge(match,on="Match_Id",how="left")

In [33]:
rw3.head()

,Match_Id,Team_Id,Team_Name,Team_Short_Code,Player_Id,Player_Name,Batsman_Scored,Dissimal_Type,Match_Date,Team_Name_Id,...,IS_Result,Is_DuckWorthLewis,Win_Type,Won_By,Match_Winner_Id,Man_Of_The_Match_Id,First_Umpire_Id,Second_Umpire_Id,City_Name,Host_Country
0,335987,1,Kolkata Knight Riders,KKR,1,SC Ganguly,20,3,18-Apr-08,2,...,1,0,by runs,140.0,1.0,2.0,470,477,Bangalore,India
1,335987,1,Kolkata Knight Riders,KKR,63,LR Shukla,9,1,18-Apr-08,2,...,1,0,by runs,140.0,1.0,2.0,470,477,Bangalore,India
2,335987,1,Kolkata Knight Riders,KKR,82,AB Agarkar,21,3,18-Apr-08,2,...,1,0,by runs,140.0,1.0,2.0,470,477,Bangalore,India
3,335987,1,Kolkata Knight Riders,KKR,84,I Sharma,6,1,18-Apr-08,2,...,1,0,by runs,140.0,1.0,2.0,470,477,Bangalore,India
4,335987,1,Kolkata Knight Riders,KKR,106,AB Dinda,7,2,18-Apr-08,2,...,1,0,by runs,140.0,1.0,2.0,470,477,Bangalore,India


In [34]:
rw3=rw3[["Match_Id","City_Name","Team_Id","Team_Name","Team_Short_Code","Player_Id","Player_Name","Batsman_Scored","Dissimal_Type"]]

In [35]:
rw3.head()

,Match_Id,City_Name,Team_Id,Team_Name,Team_Short_Code,Player_Id,Player_Name,Batsman_Scored,Dissimal_Type
0,335987,Bangalore,1,Kolkata Knight Riders,KKR,1,SC Ganguly,20,3
1,335987,Bangalore,1,Kolkata Knight Riders,KKR,63,LR Shukla,9,1
2,335987,Bangalore,1,Kolkata Knight Riders,KKR,82,AB Agarkar,21,3
3,335987,Bangalore,1,Kolkata Knight Riders,KKR,84,I Sharma,6,1
4,335987,Bangalore,1,Kolkata Knight Riders,KKR,106,AB Dinda,7,2


In [36]:
player_team=rw3[["Player_Id","Player_Name","Team_Short_Code"]]


In [37]:
pl=list(rw3["Player_Id"].unique())


In [38]:
def select_bowler(df,ls,city,team1,team2):
    a=df[df["City_Name"]==city]
    di={}
    for i in ls:
        b=a[a["Player_Id"]==i]
        Truns=b["Batsman_Scored"].sum()
        mts=len(b)
        Wtaken=b["Dissimal_Type"].sum()
        Aruns=b["Batsman_Scored"].mean()
        Mruns=b["Batsman_Scored"].median()
        MWick=b["Dissimal_Type"].median()
        Awick=b["Dissimal_Type"].mean()
        RRuns=b["Batsman_Scored"].max() -b["Batsman_Scored"].min()
        RWick=b["Dissimal_Type"].max() - b["Dissimal_Type"].min()
        std_runs=b["Batsman_Scored"].std()
        std_wick=b["Dissimal_Type"].std()
        var_run=0
        var_wick=0
        if Mruns!=0:
            var_run=std_runs/Mruns
        if MWick!=0:
            var_wick=std_wick/MWick
        di[i]={"Player_Id":i,"Total Runs Given":Truns,"Total Wickets":Wtaken,"Total Matches":mts,"Max. wkts in a match":b["Dissimal_Type"].max(),"Wts Average per match":Awick,"Wts Median per match":MWick,"Wts Range":RWick,"wts std":std_wick,"wts CV":var_wick,"Runs Average":Aruns,"Runs Median":Mruns,"Runs Range":RRuns,"Runs std":std_runs,"Runs CV":var_run}
    f=pa.DataFrame(di).T
    ff=f.merge(player_team,on="Player_Id",how="inner").drop_duplicates()
    ff= ff[(ff["Team_Short_Code"]==team1) |(ff["Team_Short_Code"]==team2)]
    ff=ff[["Player_Id","Player_Name","Team_Short_Code","Total Matches","Total Wickets","Max. wkts in a match","Wts Average per match","Wts Median per match","Wts Range","wts std","wts CV","Total Runs Given"]]
    return ff        

In [39]:
rw3.head()
tt=list(rw3["Team_Short_Code"].unique())
print(tt)

['KKR', 'RCB', 'CSK', 'KXIP', 'RR', 'DD', 'MI', 'DC', 'KTK', 'PW', 'SRH', 'RPS', 'GL']


In [43]:
pl=list(rw3["Player_Id"].unique())
rw3["City_Name"].unique()

array(['Bangalore', 'Chandigarh', 'Delhi', 'Mumbai', 'Kolkata', 'Jaipur',
       'Hyderabad', 'Chennai', 'Cape Town', 'Port Elizabeth', 'Durban',
       'Centurion', 'East London', 'Johannesburg', 'Kimberley',
       'Bloemfontein', 'Ahmedabad', 'Cuttack', 'Nagpur', 'Dharamsala',
       'Kochi', 'Indore', 'Visakhapatnam', 'Pune', 'Raipur', 'Ranchi',
       'Abu Dhabi', 'Rajkot', 'Kanpur'], dtype=object)

In [44]:

play=select_bowler(rw3,pl,"Jaipur","RR","DD")
del play["Player_Id"]
play=play[play["Total Matches"]>10]
play.reset_index()
play.sort_values("Total Wickets",ascending=False)


,Player_Name,Team_Short_Code,Total Matches,Total Wickets,Max. wkts in a match,Wts Average per match,Wts Median per match,Wts Range,wts std,wts CV,Total Runs Given
1234,SK Trivedi,RR,32.0,29.0,4.0,0.906250,0.0,4.0,1.201058,0.000000,786.0
1180,SK Warne,RR,17.0,21.0,3.0,1.235294,1.0,3.0,1.032558,1.032558,449.0
1089,SR Watson,RR,25.0,18.0,4.0,0.720000,0.0,4.0,1.061446,0.000000,572.0
4228,J Botha,RR,12.0,6.0,3.0,0.500000,0.0,3.0,0.904534,0.000000,301.0
4252,J Botha,DD,12.0,6.0,3.0,0.500000,0.0,3.0,0.904534,0.000000,301.0
